In [1]:
# S3 prefix
prefix="mt_models_uploaded/fastllm--chatglm2-6b"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
region = sagemaker_session._region_name # region name of the current environment
role = sagemaker.get_execution_role()

提前编译好ChatGLM2的Fastllm模型，上传到S3s3://${bucket}/key_prefix

In [2]:
model_s3_url = 's3://sagemaker-us-east-1-568765279027/mt_models_uploaded/fastllm-chatglm-6b-fp16/chatglm-6b-fp16.flm'

In [3]:
model_s3_url


's3://sagemaker-us-east-1-568765279027/mt_models_uploaded/fastllm-chatglm-6b-fp16/chatglm-6b-fp16.flm'

In [4]:
!docker build . -t fastllm_chatglm2

Sending build context to Docker daemon  13.25MB
Step 1/24 : FROM nvidia/cuda:11.8.0-cudnn8-devel-ubuntu20.04
 ---> 95d242fe9108
Step 2/24 : RUN apt update     && apt install -y --no-install-recommends wget python3 python3-pip     && ln -sf python3 /usr/bin/python     && ln -sf pip3 /usr/bin/pip     && pip install --upgrade pip     && pip install wheel setuptools
 ---> Using cache
 ---> cfbabcb04fda
Step 3/24 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> ff889f96d14b
Step 4/24 : RUN apt-get update     && apt-get  -y -f install build-essential gcc cmake nvidia-container-toolkit nvidia-container-runtime git
 ---> Using cache
 ---> 08cfa6047955
Step 5/24 : ARG FUNCTION_DIR="/opt/program"
 ---> Using cache
 ---> d7c0f7476c95
Step 6/24 : WORKDIR ${FUNCTION_DIR}
 ---> Using cache
 ---> 942fcc4fbf3f
Step 7/24 : RUN git clone https://github.com/ztxz16/fastllm.git ${FUNCTION_DIR}/fastllm
 ---> Using cache
 ---> 145218b00b68
Step 8/24 : WORKDIR ${FUNCTION_DIR}
 ---> Using cache
 --

In [5]:
!docker images

REPOSITORY                                                      TAG                               IMAGE ID       CREATED          SIZE
fastllm_chatglm2                                                latest                            05065f26e119   22 seconds ago   17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   latest                            d9b2860ad09e   9 minutes ago    17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            c4a408ca4451   20 minutes ago   17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            908ff925a5ab   33 hours ago     17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            8d55fadfa1f4   44 hours ago     17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            fcd234c40687   45 hours ago     17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.co

In [6]:
!docker tag fastllm_chatglm2 568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2:latest

In [7]:
!docker images

REPOSITORY                                                      TAG                               IMAGE ID       CREATED          SIZE
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   latest                            05065f26e119   26 seconds ago   17.2GB
fastllm_chatglm2                                                latest                            05065f26e119   26 seconds ago   17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            d9b2860ad09e   9 minutes ago    17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            c4a408ca4451   20 minutes ago   17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            908ff925a5ab   33 hours ago     17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2   <none>                            8d55fadfa1f4   44 hours ago     17.2GB
568765279027.dkr.ecr.us-east-1.amazonaws.co

In [8]:
repo_name = 'fastllm_chatglm2'
repo_url = f'{account_id}.dkr.ecr.{region}.amazonaws.com/fastllm_chatglm2:latest'

In [9]:
repo_name, repo_url

('fastllm_chatglm2',
 '568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2:latest')

In [10]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 568765279027.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
!aws ecr describe-repositories --repository-names $repo_name || aws ecr create-repository --repository-name $repo_name


{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:568765279027:repository/fastllm_chatglm2",
            "registryId": "568765279027",
            "repositoryName": "fastllm_chatglm2",
            "repositoryUri": "568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2",
            "createdAt": 1692164750.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


提交镜像需要运行notebook所使用的role具有相应的权限，例如：AmazonEC2ContainerRegistryFullAccess

In [12]:
!docker push 568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2:latest

The push refers to repository [568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2]

b12442e0: Preparing 
7b853bb4: Preparing 
2ee2afc1: Preparing 
46e7a12e: Preparing 
dd0e907a: Preparing 
fd852e83: Preparing 
d37e9ebb: Preparing 
48a9a225: Preparing 
e455b879: Preparing 
6ae9ac87: Preparing 
77e7f976: Preparing 
b5e06a99: Preparing 
cf377346: Preparing 
4adf2d91: Preparing 
12b2bcfe: Preparing 
a107ea4b: Preparing 
babe0dd6: Preparing 
68a70f7f: Preparing 
f07cfd83: Preparing 
654bb9f9: Preparing 
6ad9c95e: Preparing 
b4e1ecd1: Preparing 
5c845fcf: Preparing 
a7216f78: Preparing 
b853bb4: Pushed lready exists 9kB5Alatest: digest: sha256:854cb0992839b43aae48b6c18821e883361f0974604a5ffbae84b9fca887c3ed size: 5571


In [13]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, repo_name
)
print(container_image_uri)

568765279027.dkr.ecr.us-east-1.amazonaws.com/fastllm_chatglm2:latest


In [14]:

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

dummy


In [15]:
!aws s3 cp model.tar.gz s3://{bucket}/mt_models_uploaded/

upload: ./model.tar.gz to s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/model.tar.gz


In [16]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

endpoint_name = "mt-chatglm2-6b-fastllm-fp16-g4dn"

# Create the SageMaker model instance
model = Model(
    name='mt-chatglm6b-fastllm-model',
    image_uri=container_image_uri,
    role=role,
    model_data = "s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/model.tar.gz",
    env={
        "MODEL_S3_URL": model_s3_url
    }
)

model.deploy(
    instance_type='ml.g4dn.2xlarge',
    initial_instance_count=1,
    endpoint_name = endpoint_name,
)

Using already existing model: mt-chatglm6b-fastllm-model


-------------!

In [3]:
from sagemaker import serializers, deserializers

del_predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer())
# del_predictor.delete_model()
# del_predictor.delete_endpoint()
# 

In [17]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer()
)

In [18]:
# from prompt_generator import generate_prompt as gen_prompt
import importlib, sys

sys.path.append('..')

import prompt_generator as pg
importlib.reload(pg)

<module 'prompt_generator' from '/home/ec2-user/SageMaker/LLM_Workshop/fastllm/../prompt_generator.py'>

In [19]:
model_code='chatglm2'

In [38]:
json.dumps(inputs[0])

'{"prompt": "\\u5199\\u4e00\\u9996\\u5173\\u4e8e\\u4ea4\\u901a\\u4fe1\\u53f7\\u706f\\u7684\\u8bd7"}'

In [20]:
inputs= [
    {"inputs": "写一首关于交通信号灯的诗"},
    {"inputs": "陨石为什么总能落在陨石坑里?"},
    {"inputs": "为什么爸妈结婚没叫我参加婚礼?"}
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["inputs"], "\n回答:\n", response["outputs"])



问题:  写一首关于交通信号灯的诗 
回答:
  交通信号灯,闪亮如星辰,
指引行人走,车辆停。
红绿黄三种颜色,
传递安全与秩序的信息。

当红灯亮时,停车等待,
绿灯亮时,通行顺畅,
黄灯闪烁,提醒注意,
让道路通行更加安全。

交通信号灯,是我们的保障,
让交通流畅,让人民安全,
我们感激它的辛勤工作,
让它成为我们心目中的英雄。


问题:  陨石为什么总能落在陨石坑里? 
回答:
  陨石坑是因为陨石撞击地球表面而形成的环形凹坑,陨石坑内部通常会有一些水坑和岩石碎片,这是因为陨石在进入地球大气层时,由于与大气分子的摩擦而产生高温和高压,导致陨石表面发生熔融和烧蚀。

陨石坑内部由于陨石与岩石的接触而产生应力,当陨石撞击地球表面时,由于陨石的高速和力量,会使陨石表面发生变形和破裂,这些碎片会沿着陨石坑的边缘向外扩散,最终落回陨石坑内部。

此外,陨石坑内部通常会有一些水坑和岩石碎片,这是因为陨石在进入地球大气层时,由于与大气分子的摩擦而产生高温和高压,导致陨石表面发生熔融和烧蚀。水坑和岩石碎片是由于陨石表面熔融的岩石和金属在撞击过程中被溅射而形成的。

因此,陨石落在陨石坑里是由于陨石高速撞击地球表面,导致陨石表面发生变形和破裂,碎片会落回陨石坑内部,同时陨石坑内部由于陨石与岩石的接触而产生应力,使陨石表面发生熔融和烧蚀,最终形成陨石坑。


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
  父母结婚时没有叫我参加婚礼,可能是因为他们认为我年龄太小,还不适合参加这样的正式场合;也可能是他们认为我的婚礼不是很重要的,不值得他们亲自参加;再者,可能他们认为我的婚礼他们会很忙,无法抽出时间参加。

无论原因是什么,我理解他们的想法,并且感谢他们对我的关心和爱护。如果对他们的想法感到困惑或不满,可以尝试与他们沟通,了解他们的想法和原因,并尝试达成共识。


Response:   并非陨石总能落在陨石坑里,但陨石坑确实是一个常见的陨石撞击地点。陨石坑是由撞击地球表面的小行星、彗星或陨石等天体形成的。当这些天体进入地球大气层时,由于与大气分子的摩擦而产生高温和高压,导致它们开始燃烧和蒸发,形成一个明亮的火球。

当这些火球与地面接触时,由于陨石的质量和速度,它们会产生巨大的冲击力,导致地表上形成一个陨石坑。由于陨石坑本身的特征和地理位置等因素的影响,陨石坑内的陨石往往会被保存下来,并成为研究陨石撞击地球表面历史的重要样本。
Cost Time:  7.569836378097534 seconds
Output Token : 225
Speed: 29.72 token/s


In [23]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
endpoint_name = "mt-chatglm2-6b-fastllm-fp16-g4dn"

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["outputs"]
    return generated_text


In [28]:
%%time

payload = {"inputs": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n使用json格式提取：国家名，人名\n", "parameters": {"temperature": 0.01}}
payload = pg.create_payload("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", model_code)

resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

 阿根廷队伍,梅西。
CPU times: user 4.87 ms, sys: 0 ns, total: 4.87 ms
Wall time: 469 ms


In [29]:
%%time
payload = pg.create_payload("天为什么是蓝色的？", model_code)
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

 天空看起来是蓝色的主要原因是光的散射。当太阳光穿过大气层时,与空气分子相互作用,使得光产生了散射。空气分子中的气体和尘埃会散射光线,其中蓝色光的波长比红色光波长短,因此更容易被散射。

当太阳光穿过大气层时,蓝色光波比其他颜色的光波更容易被散射,而红色光波则更容易被保留。这就是为什么在日落或日出时,太阳光经过更长的大气层路径,蓝色光波的散射程度会减小,天空呈现出橙色或红色。

天空为什么是蓝色是由于光的散射,使得蓝色光波更容易被散射,而其他颜色的光波则更容易被保留。
236
CPU times: user 3.61 ms, sys: 1.73 ms, total: 5.33 ms
Wall time: 8.65 s


In [26]:
%%time
payload = pg.create_payload("小明有200元，买书花了66元，买文具花了12元，小明剩下的钱比原来少了多少元？", model_code)
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

 小明剩下的钱是 200 - 66 - 12 = 134 元。

小明剩下的钱比原来少了 66 - 134 = -50 元。
62
CPU times: user 4.61 ms, sys: 748 µs, total: 5.35 ms
Wall time: 2.73 s
